In [ ]:
from utz import *
from IPython.display import HTML, SVG, display

import fiona
import geopandas as gpd
from geopy import distance
import shapefile
from shapely.geometry import Point

import plotly.express as px
import plotly.graph_objects as go

import sys
sys.executable

## Mapbox configs

In [ ]:
token = open(".mapbox-token").read()
mapbox = dict(mapbox=dict(
    style="dark",
    accesstoken=token,
))

In [ ]:
from nj_crashes.crashes import c20
from nj_crashes.sri import get_mp_ll, SRI_DB_PATH, SRI_DB_URL, SRI_DB_TABLE

In [ ]:
grove_sri = '09061720__'
grove_crashes = c20[c20.SRI.str.contains(grove_sri)]
grove_crashes

In [ ]:
gc = grove_crashes[['Date', 'Crash Location', 'Severity', 'SRI', 'MP']]
gc

In [ ]:
grove_crashes.Severity.value_counts()

In [ ]:
gcll = sxs(gc, gc.apply(lambda r: get_mp_ll(r.SRI, r.MP), axis=1).rename('ll'))
gcll['LON'] = gcll.ll.apply(lambda p: p[0])
gcll['LAT'] = gcll.ll.apply(lambda p: p[1])
gcll = gcll.drop(columns='ll')
gcll

In [ ]:
fig = px.scatter_mapbox(
    gcll,
    lon='LON', lat='LAT',
    color='Severity',
    color_discrete_sequence=['yellow', 'orange', 'red'],
    hover_data=['Date', 'Crash Location', 'SRI', 'MP'],
    center=dict(lon=-74.042037, lat=40.725527),
    zoom=13.5,
    height=600,
)
legend_bgcolor = '50'
fig.update_layout(
    **mapbox,
    margin={"r":0,"t":0,"l":0,"b":0},
    title=dict(
        text="Grove St crashes (2020)",
        x=0.5, y=0.98,
        xanchor='center', yanchor='top',
        font=dict(size=32, color="white")
    ),
    legend=dict(
        title=dict(text=''),
        x=0.98, y=0.98,
        xanchor="right", yanchor="top",
        font=dict(
            size=14,
            color="white"
        ),
        bgcolor=f"rgba({legend_bgcolor},{legend_bgcolor},{legend_bgcolor},0.8)",
        bordercolor="white",
        borderwidth=2,
    ),
)
fig.show(config=dict(displayModeBar=False))